In [ ]:
## using optuna gives the highest r2 score YETT using xgboost
import pandas as pd
import numpy as np
import joblib
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
file_path = "dataset_true_score.xlsx"
df = pd.read_excel(file_path)

features = ["cost","pop_density", "traffic_rte", "visibility", "avg_price_level", "Comp_Score"]
target = "true_score"

# Convert target to numeric & drop missing values
df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna()

# Define X (features) and y (target)
X = df[features]
y = df[target]

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features using StandardScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define Optuna optimization function
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0)
    }
    
    model = XGBRegressor(**params, random_state=42, eval_metric="rmse",early_stopping_rounds=50)
    model.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], verbose=False)
    
    y_pred = model.predict(X_test_scaled)
    return mean_squared_error(y_test, y_pred)

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Train final model with best parameters
best_params = study.best_params
xgb = XGBRegressor(**best_params, random_state=42, eval_metric="rmse", early_stopping_rounds=50)
xgb.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], verbose=False)

# Save the trained model
joblib.dump(xgb, "xgb_model_optuna.pkl")
joblib.dump(scaler, "scaler.pkl")
print("Optimized model and scaler saved.")

# Make predictions
y_pred = xgb.predict(X_test_scaled)

# Evaluate model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Display results
print("Optimized Mean Squared Error:", mse)
print("Optimized Root Mean Squared Error:", rmse)
print("Optimized R-squared Score:", r2)

# Display sample predictions
for actual, predicted in zip(y_test[:10], y_pred[:10]):
    print(f"Actual: {actual:.4f}, Predicted: {predicted:.4f}")

[I 2025-04-04 23:23:14,155] A new study created in memory with name: no-name-6590b368-aac8-49f1-9454-0e24c9367ef6
[I 2025-04-04 23:23:14,330] Trial 0 finished with value: 2.050125813231675 and parameters: {'n_estimators': 800, 'learning_rate': 0.19090690736953414, 'max_depth': 8, 'colsample_bytree': 0.5031462784473824, 'subsample': 0.6248418399951818, 'reg_alpha': 0.08511876126123563, 'reg_lambda': 0.7278681700507614}. Best is trial 0 with value: 2.050125813231675.
[I 2025-04-04 23:23:14,554] Trial 1 finished with value: 2.015820790859772 and parameters: {'n_estimators': 550, 'learning_rate': 0.11040696375878975, 'max_depth': 8, 'colsample_bytree': 0.8275229006849196, 'subsample': 0.8080147830793789, 'reg_alpha': 0.13431841455346616, 'reg_lambda': 0.980806493561907}. Best is trial 1 with value: 2.015820790859772.
[I 2025-04-04 23:23:14,967] Trial 2 finished with value: 2.0173399217079866 and parameters: {'n_estimators': 250, 'learning_rate': 0.08042620280159227, 'max_depth': 8, 'colsam

Optimized model and scaler saved.
Optimized Mean Squared Error: 1.984845144418669
Optimized Root Mean Squared Error: 1.4088453230992637
Optimized R-squared Score: 0.08329821935238524
Actual: 4.4782, Predicted: 2.5456
Actual: 3.9231, Predicted: 2.4743
Actual: 0.5914, Predicted: 2.9160
Actual: 0.9616, Predicted: 2.0083
Actual: 4.2000, Predicted: 3.2595
Actual: 4.1915, Predicted: 3.5868
Actual: 1.3910, Predicted: 3.4000
Actual: 3.4682, Predicted: 2.8803
Actual: 3.8512, Predicted: 2.8943
Actual: 3.4591, Predicted: 3.1604


In [9]:
#optuna lgbm
import pandas as pd
import numpy as np
import joblib
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
file_path = "dataset_true_score.xlsx"
df = pd.read_excel(file_path)

# Define features and target
features = ["cost", "pop_density", "traffic_rte", "visibility", "avg_price_level", "Comp_Score"]
target = "true_score"

# Convert target to numeric & drop missing values
df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna()

# Define X and y
X = df[features]
y = df[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define Optuna objective function
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0)
    }
    
    model = LGBMRegressor(**params, random_state=42,early_stopping_rounds=50)
    model.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)],
                callbacks=[])
    
    y_pred = model.predict(X_test_scaled)
    return mean_squared_error(y_test, y_pred)

# Run Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Train final model with best parameters
best_params = study.best_params
lgbm = LGBMRegressor(**best_params, random_state=42,early_stopping_rounds=50)
lgbm.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)],
          callbacks=[])

# Save model and scaler
joblib.dump(lgbm, "lgbm_model_optuna.pkl")
joblib.dump(scaler, "scaler.pkl")
print("Optimized LGBM model and scaler saved.")

# Predictions
y_pred = lgbm.predict(X_test_scaled)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Optimized Mean Squared Error:", mse)
print("Optimized Root Mean Squared Error:", rmse)
print("Optimized R-squared Score:", r2)

# Show sample predictions
for actual, predicted in zip(y_test[:10], y_pred[:10]):
    print(f"Actual: {actual:.4f}, Predicted: {predicted:.4f}")

[I 2025-04-04 22:56:27,685] A new study created in memory with name: no-name-125e00a1-5502-4639-b3d1-59e66c61070e


[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000389 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1105
[LightGBM] [Info] Number of data points in the train set: 25027, number of used features: 6
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Start training from score 3.006600
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

/home/rohanj48/code/major/backend/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-04-04 22:56:27,919] Trial 0 finished with value: 2.0385900764337284 and parameters: {'n_estimators': 650, 'learning_rate': 0.011075095757220647, 'max_depth': 3, 'num_leaves': 137, 'subsample': 0.7987850926460395, 'colsample_bytree': 0.7360939486492064, 'reg_alpha': 0.5886237802425643, 'reg_lambda': 0.8828198687791134}. Best is trial 0 with value: 2.0385900764337284.
/home/rohanj48/code/major/backend/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-04-04 22:56:28,068] Trial 1 finished with value: 2.0269734972997826 and parameters: {'n_estimators': 950, 'learning_rate': 0.09685387626360314, 'max_depth': 4, 'num_leaves': 63, 'su

[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000264 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1105
[LightGBM] [Info] Number of data points in the train set: 25027, number of used features: 6
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Start training from score 3.006600
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits wi

/home/rohanj48/code/major/backend/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-04-04 22:56:28,317] Trial 2 finished with value: 2.0176343225380617 and parameters: {'n_estimators': 1000, 'learning_rate': 0.049103571923716616, 'max_depth': 7, 'num_leaves': 50, 'subsample': 0.8842225951259082, 'colsample_bytree': 0.9726584952103241, 'reg_alpha': 0.15323783278740233, 'reg_lambda': 0.9980447572987893}. Best is trial 2 with value: 2.0176343225380617.


...

/home/rohanj48/code/major/backend/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-04-04 22:56:47,255] Trial 48 finished with value: 2.006449448791908 and parameters: {'n_estimators': 950, 'learning_rate': 0.022323273176813305, 'max_depth': 12, 'num_leaves': 135, 'subsample': 0.6510973459584889, 'colsample_bytree': 0.764407929982302, 'reg_alpha': 0.48355172421930565, 'reg_lambda': 0.5846978980937738}. Best is trial 45 with value: 2.0012992721116314.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/home/rohanj48/code/major/backend/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-04-04 22:56:48,541] Trial 49 finished with value: 2.006564500890695 and parameters: {'n_estimators': 850, 'learning_rate': 0.011904989982508192, 'max_depth': 12, 'num_leaves': 147, 'subsample': 0.564522195369332, 'colsample_bytree': 0.8134952636925892, 'reg_alpha': 0.5359049456539752, 'reg_lambda': 0.6216043345049393}. Best is trial 45 with value: 2.0012992721116314.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iter

/home/rohanj48/code/major/backend/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [2]:
## using optuna gives the highest r2 score YETT using xgboost
import pandas as pd
import numpy as np
import joblib
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
file_path = "augmented_dataset_true_score.xlsx"
df = pd.read_excel(file_path)


features = ["cost","pop_density", "traffic_rte", "visibility", "avg_price_level", "Comp_Score"]
target = "true_score"

# Convert target to numeric & drop missing values
df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna()

# Define X (features) and y (target)
X = df[features]
y = df[target]

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features using StandardScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define Optuna optimization function
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0)
    }
    
    model = XGBRegressor(**params, random_state=42, eval_metric="rmse",early_stopping_rounds=50)
    model.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], verbose=False)
    
    y_pred = model.predict(X_test_scaled)
    return mean_squared_error(y_test, y_pred)

# Run Optuna optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Train final model with best parameters
best_params = study.best_params
xgb = XGBRegressor(**best_params, random_state=42, eval_metric="rmse", early_stopping_rounds=50)
xgb.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], verbose=False)

# Save the trained model
joblib.dump(xgb, "xgb_model_optuna.pkl")
joblib.dump(scaler, "scaler.pkl")
print("Optimized model and scaler saved.")

# Make predictions
y_pred = xgb.predict(X_test_scaled)

# Evaluate model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Display results
print("Optimized Mean Squared Error:", mse)
print("Optimized Root Mean Squared Error:", rmse)
print("Optimized R-squared Score:", r2)

# Display sample predictions
for actual, predicted in zip(y_test[:10], y_pred[:10]):
    print(f"Actual: {actual:.4f}, Predicted: {predicted:.4f}")

[I 2025-04-04 22:40:36,970] A new study created in memory with name: no-name-8c89dedd-d44e-4b52-8bfd-9e7c188b47a1
[I 2025-04-04 22:40:37,865] Trial 0 finished with value: 1.445382394280455 and parameters: {'n_estimators': 600, 'learning_rate': 0.017989490012315052, 'max_depth': 6, 'colsample_bytree': 0.717263785503407, 'subsample': 0.5463071234674459, 'reg_alpha': 0.7204837011089145, 'reg_lambda': 0.5152355056769679}. Best is trial 0 with value: 1.445382394280455.
[I 2025-04-04 22:40:38,115] Trial 1 finished with value: 1.422107008780122 and parameters: {'n_estimators': 800, 'learning_rate': 0.013801271735133546, 'max_depth': 7, 'colsample_bytree': 0.9592784775198833, 'subsample': 0.5185873088667242, 'reg_alpha': 0.42498427954197227, 'reg_lambda': 0.05889666983861619}. Best is trial 1 with value: 1.422107008780122.
[I 2025-04-04 22:40:38,899] Trial 2 finished with value: 1.4096344562572305 and parameters: {'n_estimators': 500, 'learning_rate': 0.056514996289704425, 'max_depth': 6, 'col

Optimized model and scaler saved.
Optimized Mean Squared Error: 1.3651221858185985
Optimized Root Mean Squared Error: 1.16838443408777
Optimized R-squared Score: 0.37754879529145613
Actual: 4.2940, Predicted: 3.0451
Actual: 2.3458, Predicted: 2.3917
Actual: 1.7798, Predicted: 1.6675
Actual: 0.6163, Predicted: 1.2621
Actual: 3.9000, Predicted: 3.6966
Actual: 3.2552, Predicted: 2.2994
Actual: 4.1000, Predicted: 3.5712
Actual: 1.2580, Predicted: 1.7010
Actual: 1.7330, Predicted: 1.3728
Actual: 4.6000, Predicted: 3.8116


In [2]:
import pandas as pd
import numpy as np
import joblib
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
file_path = "augmented_dataset_true_score.xlsx"
df = pd.read_excel(file_path)

# Define features and target
features = ["cost", "pop_density", "traffic_rte", "visibility", "avg_price_level", "Comp_Score"]
target = "true_score"

# Convert target to numeric & drop missing values
df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna()

# Define X and y
X = df[features]
y = df[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define Optuna objective function
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "num_leaves": trial.suggest_int("num_leaves", 20, 150),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 1.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 1.0)
    }
    
    model = LGBMRegressor(**params, random_state=42,early_stopping_rounds=50)
    model.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)],
                callbacks=[])
    
    y_pred = model.predict(X_test_scaled)
    return mean_squared_error(y_test, y_pred)

# Run Optuna
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Train final model with best parameters
best_params = study.best_params
lgbm = LGBMRegressor(**best_params, random_state=42,early_stopping_rounds=50)
lgbm.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)],
          callbacks=[])

# Save model and scaler
joblib.dump(lgbm, "lgbm_model_optuna.pkl")
joblib.dump(scaler, "scaler.pkl")
print("Optimized LGBM model and scaler saved.")

# Predictions
y_pred = lgbm.predict(X_test_scaled)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Optimized Mean Squared Error:", mse)
print("Optimized Root Mean Squared Error:", rmse)
print("Optimized R-squared Score:", r2)

# Show sample predictions
for actual, predicted in zip(y_test[:10], y_pred[:10]):
    print(f"Actual: {actual:.4f}, Predicted: {predicted:.4f}")


[I 2025-04-06 17:23:16,287] A new study created in memory with name: no-name-9cd54aae-47ba-4097-8036-fd89286ee5d5


[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000916 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 52735, number of used features: 6
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Start training from score 2.088721
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

/home/rohanj48/code/major/backend/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-04-06 17:23:17,446] Trial 0 finished with value: 1.3840140616458776 and parameters: {'n_estimators': 650, 'learning_rate': 0.019023552427657093, 'max_depth': 11, 'num_leaves': 81, 'subsample': 0.5967037290640417, 'colsample_bytree': 0.5710500912596709, 'reg_alpha': 0.6264289376493425, 'reg_lambda': 0.9796580142923667}. Best is trial 0 with value: 1.3840140616458776.
/home/rohanj48/code/major/backend/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-04-06 17:23:17,606] Trial 1 finished with value: 1.4628412913696127 and parameters: {'n_estimators': 300, 'learning_rate': 0.05152790937043831, 'max_depth': 3, 'num_leaves': 25, 'su

[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000782 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 52735, number of used features: 6
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Start training from score 2.088721
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

/home/rohanj48/code/major/backend/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-04-06 17:23:17,860] Trial 2 finished with value: 1.4882978694546511 and parameters: {'n_estimators': 200, 'learning_rate': 0.017038734706100456, 'max_depth': 6, 'num_leaves': 120, 'subsample': 0.9028664498115972, 'colsample_bytree': 0.9898743905953404, 'reg_alpha': 0.46162600607537196, 'reg_lambda': 0.07102271600437171}. Best is trial 0 with value: 1.3840140616458776.


...

/home/rohanj48/code/major/backend/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-04-06 17:23:58,857] Trial 48 finished with value: 1.3577088742922403 and parameters: {'n_estimators': 650, 'learning_rate': 0.07839686884597474, 'max_depth': 11, 'num_leaves': 85, 'subsample': 0.7617699618560149, 'colsample_bytree': 0.9770785820551854, 'reg_alpha': 0.7386478804034725, 'reg_lambda': 0.1368181897006921}. Best is trial 29 with value: 1.3554432770214284.


Did not meet early stopping. Best iteration is:
[636]	valid_0's l2: 1.35771
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000803 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 52735, number of used features: 6
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Start training from score 2.088721
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furth

/home/rohanj48/code/major/backend/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
[I 2025-04-06 17:23:59,611] Trial 49 finished with value: 1.3646966666311782 and parameters: {'n_estimators': 750, 'learning_rate': 0.08316390907745748, 'max_depth': 12, 'num_leaves': 86, 'subsample': 0.7609691893863579, 'colsample_bytree': 0.9524776001002138, 'reg_alpha': 0.7406490399426696, 'reg_lambda': 0.13396182060809905}. Best is trial 29 with value: 1.3554432770214284.


Early stopping, best iteration is:
[393]	valid_0's l2: 1.3647
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000113 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 52735, number of used features: 6
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Start training from score 2.088721
Training until validation scores don't improve for 50 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive g

/home/rohanj48/code/major/backend/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [4]:
# random forest
import pandas as pd
import numpy as np
import joblib
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
file_path = "dataset_true_score.xlsx"
df = pd.read_excel(file_path)

features = ["cost", "pop_density", "traffic_rte", "visibility", "avg_price_level", "Comp_Score"]
target = "true_score"

# Clean data
df[target] = pd.to_numeric(df[target], errors='coerce')
df = df.dropna()

X = df[features]
y = df[target]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Optuna optimization
def objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "max_depth": trial.suggest_int("max_depth", 5, 30),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 4),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),
    }

    model = RandomForestRegressor(**params, random_state=42)
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    return -r2_score(y_test, y_pred)  # Negative because Optuna minimizes

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=50)

# Train best model
best_params = study.best_params
rf = RandomForestRegressor(**best_params, random_state=42)
rf.fit(X_train_scaled, y_train)

# Save model
joblib.dump(rf, "rf_model_optuna.pkl")
joblib.dump(scaler, "scaler.pkl")
print("Optimized RF model and scaler saved.")

# Predictions
y_pred = rf.predict(X_test_scaled)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print("Optimized Mean Squared Error:", mse)
print("Optimized Root Mean Squared Error:", rmse)
print("Optimized R-squared Score:", r2)

# Show predictions
for actual, predicted in zip(y_test[:10], y_pred[:10]):
    print(f"Actual: {actual:.4f}, Predicted: {predicted:.4f}")


[I 2025-04-04 23:24:44,225] A new study created in memory with name: no-name-2d0d30a1-bdf2-4248-9d47-3c4e6093c20c
[I 2025-04-04 23:24:53,646] Trial 0 finished with value: -0.07939828098378154 and parameters: {'n_estimators': 400, 'max_depth': 15, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': 'log2'}. Best is trial 0 with value: -0.07939828098378154.
[I 2025-04-04 23:25:14,077] Trial 1 finished with value: -0.07800935783260199 and parameters: {'n_estimators': 1000, 'max_depth': 13, 'min_samples_split': 3, 'min_samples_leaf': 1, 'max_features': 'sqrt'}. Best is trial 0 with value: -0.07939828098378154.
[I 2025-04-04 23:25:36,216] Trial 2 finished with value: -0.08387594415131383 and parameters: {'n_estimators': 800, 'max_depth': 22, 'min_samples_split': 7, 'min_samples_leaf': 4, 'max_features': 'log2'}. Best is trial 2 with value: -0.08387594415131383.
[I 2025-04-04 23:25:51,028] Trial 3 finished with value: -0.08796235261066776 and parameters: {'n_estimators': 500, 'max

Optimized RF model and scaler saved.
Optimized Mean Squared Error: 1.9684610008679293
Optimized Root Mean Squared Error: 1.4030185319046677
Optimized R-squared Score: 0.09086524472440638
Actual: 4.4782, Predicted: 2.2540
Actual: 3.9231, Predicted: 2.2994
Actual: 0.5914, Predicted: 3.0317
Actual: 0.9616, Predicted: 2.1452
Actual: 4.2000, Predicted: 3.2190
Actual: 4.1915, Predicted: 3.5262
Actual: 1.3910, Predicted: 3.3515
Actual: 3.4682, Predicted: 2.5866
Actual: 3.8512, Predicted: 3.0822
Actual: 3.4591, Predicted: 3.1524
